In [3]:
from graph import TechnicalAnalysis
import os

In [3]:
csv_files = [os.path.join(os.getcwd(), "raw_data", file) for file in os.listdir("raw_data")]

NameError: name 'os' is not defined

In [6]:
file_path = [file for file in file_paths if file.endswith('TCS_data.csv')]
file_path

['c:\\Users\\arsha\\OneDrive\\Desktop\\Stockify_model_v0.1\\stockify\\Stockify\\raw_data\\TCS_data.csv']

In [7]:
ta = TechnicalAnalysis(file_path)
ta.analyze_rsi(file_path[0])

In [1]:
import json

In [5]:
result_data = json.load(open('Output/result_data.json', 'r'))
sentiments = json.load(open('Output/news_sentiment.json', 'r'))

In [6]:
result_data

[{'stock_ticker': 'HDFCBANK_data',
  'current_price': 1640.5,
  'recommendation': '0.3%',
  'accuracy': '84.0%',
  'confidence_interval': [4618, 5219]},
 {'stock_ticker': 'LT_data',
  'current_price': 2622.15,
  'recommendation': '-4.16%',
  'accuracy': '86.0%',
  'confidence_interval': [33076, 38355]},
 {'stock_ticker': 'TCS_data',
  'current_price': 3440.65,
  'recommendation': '-0.34%',
  'accuracy': '91.0%',
  'confidence_interval': [14006, 15998]},
 {'stock_ticker': 'TITAN_data',
  'current_price': 2977.15,
  'recommendation': '-1.67%',
  'accuracy': '90.0%',
  'confidence_interval': [33395, 38058]}]

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.express as px
import plotly.graph_objects as go

from copy import copy
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout


from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import t

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def LSTM_model( timestamp=6):
        
        
# Define a new create_dataset function to create target sequences with the same number of features as the input sequences
        def X_Y(dataset, time_step=1):
            dataX, dataY = [], []
            for i in range(len(dataset) - time_step):
                a = dataset[i:(i + time_step), :]
                dataX.append(a)
                dataY.append(dataset[i + time_step, :])  # Append entire row instead of just one element
            return np.array(dataX), np.array(dataY)
        
        
       
        
        

        def get_recommendation(model, xtest, ytest, scaler, last_day_price):
            pred = model.predict(xtest)
            
            # Get the last 'time_step' number of data points from the test set
            last_sequence = X_test[-1]

            # Reshape the data to match the input shape of the model
            last_sequence = last_sequence.reshape(1, last_sequence.shape[0], last_sequence.shape[1])

            # Use the model to predict the next day's 'Open' and 'Close' prices
            next_day_pred = model.predict(last_sequence)

            # Inverse scale the predicted values to get the prices in the original data scale
            next_day_pred_original_scale = scaler.inverse_transform(next_day_pred)


            
            next_hour_open_price = next_day_pred_original_scale[0, 0]  
            next_hour_high_price = next_day_pred_original_scale[0, 1] 
            next_hour_low_price = next_day_pred_original_scale[0, 2] 
            next_hour_close_price = next_day_pred_original_scale[0, 3] 

#             print("Predicted Next hour's Open Price:", next_hour_open_price)
#             print("Predicted Next hour's high Price:", next_hour_high_price)
#             print("Predicted Next hour's low Price:", next_hour_low_price)
#             print("Predicted Next hour's Close Price:", next_hour_close_price)

            accuracy = r2_score(ytest, pred)
            current_price = last_day_price[-1]
            
            
            
            recommendation = ((next_hour_close_price - current_price) / current_price) * 100

            return current_price, recommendation, accuracy,next_hour_close_price
        
        result_data = []
        for file_path in csv_files:
            
            input_filename = os.path.basename(file_path).split(".")[0]
            
            df = pd.read_csv(file_path)
            df['Datetime'] = pd.to_datetime(df['Datetime']).dt.date
            
            closedf = df[[ 'Open', 'High', 'Low', 'Close']]

            # Normalize all columns in the DataFrame except for 'Date'
            scaler = MinMaxScaler(feature_range=(0, 1))
            closedf = scaler.fit_transform(closedf)
            print(closedf.shape)  

            
            
            training_size = int(len(closedf) * 0.70)
            test_size = len(closedf) - training_size
            
            train_data = closedf[0:training_size, :]
            test_data = closedf[training_size:len(closedf), :]
            
            time_step = 6
            X_train, y_train = X_Y(train_data, time_step)
            X_test, y_test = X_Y(test_data, time_step)
            
            

            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 4)
            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 4)
            
            
            # # Assuming you have already defined other variables like X_train, y_train, X_test, and y_test

            # Clear any existing models from memory
            tf.keras.backend.clear_session()

            # Initialize the Sequential model
            model = Sequential()

            # Add LSTM layers with dropout
            model.add(LSTM(32, return_sequences=True, input_shape=(time_step, 4)))
            model.add(Dropout(0.2))  # Add dropout with 20% probability of setting input units to zero
            model.add(LSTM(32, return_sequences=True))
            model.add(Dropout(0.2))  # Add dropout with 20% probability of setting input units to zero
            model.add(LSTM(32))
            model.add(Dropout(0.2))  # Add dropout with 20% probability of setting input units to zero

            # Add the Dense output layer
            model.add(Dense(4))

            # Compile the model
            model.compile(loss='mean_squared_error', optimizer='adam')


            history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64, verbose=1)

            current_price, recommendation, accuracy,next_hour_open_price = get_recommendation(model, X_test, y_test, scaler, df['Close'].values)
            result_data.append({
                "stock_ticker": input_filename,
                "current_price": round(current_price,2),
                "recommendation":  f"{round(recommendation,2)}%",
                "accuracy": f"{round(accuracy,2)*100}%",
                "next_hour_open_price":next_hour_open_price
            })
            
#         json.dump(result_data, open('Output/result_data.json', 'w'))
        print(result_data)
        return result_data